In [6]:
import os
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import numpy as np
import json
import requests
from tqdm import tqdm
from parse import *
import re
import math

In [7]:
texts_df = pd.read_parquet('parsed_texts.parquet')
texts_n = len(texts_df)
texts_n

450

In [8]:
for i in range(texts_n):
    texts_df.iloc[i]['text'] = texts_df.iloc[i]['text'].replace(u'\xa0', u' ').replace('\t', ' ').strip()
    while '\n\n' in texts_df.iloc[i]['text']:
        texts_df.iloc[i]['text'] = texts_df.iloc[i]['text'].replace('\n\n', '\n')

In [9]:
print(texts_df.iloc[252]['text'])

О проведении общего собрания участников (акционеров) эмитента и о принятых им решениях
1. Общие сведения
1.1. Полное фирменное наименование эмитента: Закрытое Акционерное Общество "ЖБИ-2"
1.2. Сокращенное фирменное наименование эмитента: ЗАО "ЖБИ-2"
1.3. Место нахождения эмитента: Россия, 454010  г. Челябинск, Копейское шоссе, 50
1.4. ОГРН эмитента: 1027402694648
1.5. ИНН эмитента: 7449010303
1.6. Уникальный код эмитента, присвоенный регистрирующим органом: NA
1.7. Адрес страницы в сети Интернет, используемой эмитентом для раскрытия информации: http://www.disclosure.ru/issuer/7449010303/
1.8. Дата наступления события (существенного факта), о котором составлено сообщение (если применимо): 30.11.2020
2. Содержание сообщения
Форма проведения общего собрания участников (акционеров) эмитента:  заочное голосование Почтовый адрес, по которому должны направляться бюллетени для голосования : Россия, 454010 г. Челябинск, Копейское шоссе, 50, канцелярия ЗАО «ЖБИ-2»
Дата, место, время проведения о

In [10]:
invalid_texts = {
    77: {
        'full_name': 'Акционерное общество «Омский каучук»',
        'short_name': 'АО «Омский каучук»',
        'address': '644035, г. Омск, пр-кт Губкина, 30',
        'inn': '5501023216',
        'ogrn': '1025500520297',
        'date': '21.12.2020',
        'form': 'заочное голосование',
        'director': 'Н.П. Комаров',
        'dividends': 'принято решение выплатить дивиденды'
    },
    151: {
        'full_name': 'Открытое акционерное общество «Завод им. В.А. Дегтярева»',
        'short_name': 'ОАО «ЗиД»',
        'address': 'Российская Федерация, Владимирская обл., г. Ковров, ул. Труда,  4',
        'inn': '3305004083',
        'ogrn': '1023301951397',
        'date': '11.12.2020',
        'form': 'заочное голосование',
        'director': 'А.В.  Тменов',
        'dividends': 'принято решение выплатить дивиденды'
    },
    170: {
        'full_name': 'Публичное акционерное общество «Горно-металлургическая компания «Норильский никель»',
        'short_name': 'ПАО «ГМК «Норильский никель»',
        'address': 'Российская Федерация, Красноярский край, г. Дудинка',
        'inn': '8401005730',
        'ogrn': '1028400000298',
        'date': '11.12.2020',
        'form': 'тип собрания не определён',
        'director': 'Н.Ю. Юрченко',
        'dividends': 'принято решение выплатить дивиденды'
    },
    300: {
        'full_name': 'открытое акционерное общество "Студенецкий мукомольный завод"',
        'short_name': 'ОАО "Студенецкий мукомольный завод"',
        'address': '442227, Пензенская область, Каменский район, ст. Студенец, ул. Станционная, 1',
        'inn': '5815000308',
        'ogrn': '1025800507787',
        'date': '18.11.2020',
        'form': 'тип собрания не определён',
        'director': 'Ужов А.В.',
        'dividends': 'вопрос не поднимался'
    },
    301: {
        'full_name': 'открытое акционерное общество "Студенецкий мукомольный завод"',
        'short_name': 'ОАО "Студенецкий мукомольный завод"',
        'address': '442227, Пензенская область, Каменский район, ст. Студенец, ул. Станционная, 1',
        'inn': '5815000308',
        'ogrn': '1025800507787',
        'date': '17.11.2020',
        'form': 'совместное присутствие',
        'director': None,
        'dividends': 'вопрос не поднимался'
    },
    311: {
        'full_name': 'Открытое акционерное общество «Юнибанк»',
        'short_name': 'ОАО «Юнибанк»',
        'address': 'Республика Армения, г. Ереван, 0025, улица Чаренца 12, No 53, 1-5',
        'inn': None,
        'ogrn': None,
        'date': '13.11.2020',
        'form': 'тип собрания не определён',
        'director': None,
        'dividends': 'вопрос не поднимался'
    },
    345: {
        'full_name': 'Акционерное общество "Мордовская ипотечная корпорация"',
        'short_name': 'АО "МИК"',
        'address': 'РФ, Республика Мордовия , г.Саранск , ул. Коммунистическая ,д. 89',
        'inn': '1326183513',
        'ogrn': '1021300979127',
        'date': '03.11.2020',
        'form': 'тип собрания не определён',
        'director': None,
        'dividends': 'вопрос не поднимался'
    },
    346: {
        'full_name': 'Акционерное общество "Мордовская ипотечная корпорация"',
        'short_name': 'АО "МИК"',
        'address': 'РФ, Республика Мордовия , г.Саранск , ул. Коммунистическая ,д. 89',
        'inn': '1326183513',
        'ogrn': '1021300979127',
        'date': '03.11.2020',
        'form': 'тип собрания не определён',
        'director': None,
        'dividends': 'вопрос не поднимался'
    },
    360: {
        'full_name': 'Открытое Акционерное Общество "СТАНДАРТЭЛЕКТРО"',
        'short_name': 'ОАО "СТАНДАРТЭЛЕКТРО"',
        'address': '123242, г. Москва, ул. Зоологическая, д.1, стр.1',
        'inn': '7710037199',
        'ogrn': '1027700070683',
        'date': '30.10.2020',
        'form': 'тип собрания не определён',
        'director': 'КОМАРОВА Е.Е.',
        'dividends': 'вопрос не поднимался'
    },
    361: {
        'full_name': 'Открытое Акционерное Общество "СТАНДАРТЭЛЕКТРО"',
        'short_name': 'ОАО "СТАНДАРТЭЛЕКТРО"',
        'address': '123242, г. Москва, ул. Зоологическая, д.1, стр.1',
        'inn': '7710037199',
        'ogrn': '1027700070683',
        'date': '30.10.2020',
        'form': 'тип собрания не определён',
        'director': None,
        'dividends': 'вопрос не поднимался'
    },
    371: {
        'full_name': 'АКЦИОНЕРНОЕ ОБЩЕСТВО "РОСТОВСКИЙ ПОРТ"',
        'short_name': 'АО "РОСТОВСКИЙ ПОРТ"',
        'address': 'г.Ростов-на-Дону, ул.Береговая, 30',
        'inn': '6167014330',
        'ogrn': '1026104143780',
        'date': '30.10.2020',
        'form': 'тип собрания не определён',
        'director': 'Швацкий Павел Алексеевич',
        'dividends': 'вопрос не поднимался'
    }
}

In [11]:
def get_text_blocks(text):
    reg_first = re.compile('1\.1\.[\s\S]+?(?=\n2)', re.M)
    reg_second = re.compile('2\..+ения[\s\S]+?(?=\n3)', re.M)
    reg_third = re.compile('3\.1\.?\D[\s\S]*', re.M)
    
    return (re.findall(reg_first, text)[0].strip(), re.findall(reg_second, text)[0].strip(), re.findall(reg_third, text)[0].strip())

## Полное наименование эмитента

In [12]:
end_words = ('ание)', ':', 'эмитента')

def get_full_name(text, index):
    if index in invalid_texts:
        return invalid_texts[index]['full_name']
    
    first_block = get_text_blocks(text)[0]
    curr = re.split('\s', re.findall('1\.1\.[\s\S]+(?=\n1\.2\.)', first_block)[0])
    result = ''
    
    for i in range(len(curr) - 1, -1, -1):
        if curr[i].endswith(end_words):
            break
        else:
            result = curr[i] + ' ' + result
    
    return result.strip()

In [13]:
full_names = []

for i in range(len(texts_df)):
    full_names.append(get_full_name(texts_df.iloc[i]['text'], i))

In [14]:
full_names_df = pd.DataFrame(full_names, columns=['Полное фирменное наименование эмитента'])
full_names_df

,Полное фирменное наименование эмитента
0,Иркутское публичное акционерное общество энерг...
1,"Публичное акционерное общество ""Новороссийский..."
2,"НЕПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""ЦЕНТРТРУБ..."
3,"НЕПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""ЦЕНТРТРУБ..."
4,Публичное акционерное общество «Русская Акваку...
...,...
445,"Публичное акционерное общество ""Акционерный го..."
446,"Публичное акционерное общество ""Челябинский ме..."
447,"Публичное акционерное общество ""САФМАР Финансо..."
448,"Акционерное общество ""МаксимаТелеком"""


## Сокращенное наименование эмитента

In [15]:
invalid_short_names = {
    18: 'ПАО «Роствертол»'
}

In [16]:
end_words = (':', 'ента')

def get_short_name(text, index):
    if index in invalid_texts:
        return invalid_texts[index]['short_name']
    elif index in invalid_short_names:
        return invalid_short_names[index]
    
    first_block = get_text_blocks(text)[0]
    curr = re.split('\s', re.findall('1\.2\.[\s\S]+(?=\n1[. ]3\.)', first_block)[0])
    result = ''
    
    for i in range(len(curr) - 1, -1, -1):
        if curr[i].endswith(end_words):
            break
        else:
            result = curr[i] + ' ' + result
    
    return result.strip()

In [17]:
short_names = []

for i in range(len(texts_df)):
    short_names.append(get_short_name(texts_df.iloc[i]['text'], i))

In [18]:
short_names_df = pd.DataFrame(short_names, columns=['Сокращенное наименование эмитента'])
short_names_df

,Сокращенное наименование эмитента
0,"ПАО ""Иркутскэнерго"""
1,"ПАО ""НКХП"""
2,"АО ""ЦЕНТРТРУБОПРОВОДСТРОЙ"""
3,"АО ""ЦЕНТРТРУБОПРОВОДСТРОЙ"""
4,ПАО «Русская Аквакультура»
...,...
445,"ПАО ""Таганрогбанк"""
446,"ПАО ""ЧМК"""
447,"ПАО ""САФМАР Финансовые инвестиции"""
448,"АО ""МаксимаТелеком"""


## Адрес

In [19]:
end_words = (':', 'ента')


def get_address(text, index):
    if index in invalid_texts:
        return invalid_texts[index]['address']
    
    first_block = get_text_blocks(text)[0]
    curr = re.split('\s', re.findall('1[. ]3\.[\s\S]+(?=\n1\.4\.)', first_block)[0])
    result = ''
    
    for i in range(len(curr) - 1, -1, -1):
        if curr[i].endswith(end_words):
            break
        else:
            result = curr[i] + ' ' + result
    
    return result.strip()

In [20]:
addresses = []

for i in range(len(texts_df)):
    addresses.append(get_address(texts_df.iloc[i]['text'], i))

In [21]:
addresses_df = pd.DataFrame(addresses, columns=['Адрес эмитента'])
addresses_df

,Адрес эмитента
0,"664025, г. Иркутск, ул. Сухэ-Батора, 3"
1,"353901, Российская Федерация, Краснодарский кр..."
2,"101000, Г.МОСКВА, ПЕРЕУЛОК КРИВОКОЛЕННЫЙ, ДО..."
3,"101000, Г.МОСКВА, ПЕРЕУЛОК КРИВОКОЛЕННЫЙ, ДО..."
4,"Россия, 121353, г. Москва, ул. Беловежская, д. 4"
...,...
445,"347900, Ростовская область, г. Таганрог, ул. Г..."
446,"Российская Федерация, 454047, г. Челябинск, ул..."
447,"Российская Федерация, г. Москва"
448,"Российская Федерация, город Москва, Старопимен..."


## ОГРН

In [22]:
def get_ogrn(text, index):
    if index in invalid_texts:
        return invalid_texts[index]['ogrn']
    
    first_block = get_text_blocks(text)[0]
    result = re.findall('\d{13}', first_block)[0]
    
    return result

In [23]:
ogrns = []

for i in range(len(texts_df)):
    ogrns.append(get_ogrn(texts_df.iloc[i]['text'], i))

In [24]:
ogrns_df = pd.DataFrame(ogrns, columns=['ОГРН эмитента'])
ogrns_df

,ОГРН эмитента
0,1023801003313
1,1032309077822
2,1027739082062
3,1027739082062
4,1079847122332
...,...
445,1026100002565
446,1027402812777
447,1027700085380
448,1047796902450


## ИНН

In [25]:
def get_inn(text, index):
    if index in invalid_texts:
        return invalid_texts[index]['inn']
    
    first_block = get_text_blocks(text)[0]
    result = re.findall('\s\d{10}\s', first_block)[0].strip()
    
    return result

In [26]:
inns = []

for i in range(len(texts_df)):
    inns.append(get_inn(texts_df.iloc[i]['text'], i))

In [27]:
inns_df = pd.DataFrame(inns, columns=['ИНН эмитента'])
inns_df

,ИНН эмитента
0,3800000220
1,2315014748
2,7701012060
3,7701012060
4,7816430057
...,...
445,6154035357
446,7450001007
447,6164077483
448,7703534295


## Дата собрания

In [28]:
invalid_dates = {
    11: '30.12.2020',
    77: '24.12.2020',
    198: '07.12.2020',
    303: '17.11.2020'
}

In [29]:
def beautify_date(d):
    d = re.sub(r'[“”«»"]', '', d)
    d = re.sub(r'(\d{2})\s(\d{2})', r'\1\2', d)
    while '  ' in d:
        d = d.replace('  ', ' ')
    return d.strip()

In [30]:
end_words = (':', 'ата', '3.2.')


def get_date(text, index):
    if index in invalid_texts:
        return invalid_texts[index]['date']
    elif index in invalid_dates:
        return invalid_dates[index]
    
    third_block = get_text_blocks(text)[2]
    curr = re.split('\s', re.findall('3\.2\.[\s\S]+', third_block)[0])
    result = ''
        
    for i in range(len(curr) - 1, -1, -1):
        if curr[i].endswith(end_words):
            break
        elif not curr[i].startswith(('М', 'П')):
            result = curr[i] + ' ' + result
    
    return beautify_date(result)

In [31]:
dates = []

for i in range(len(texts_df)):
    dates.append(get_date(texts_df.iloc[i]['text'], i))

In [32]:
dates_df = pd.DataFrame(dates, columns=['Дата собрания'])
dates_df

,Дата собрания
0,31.12.2020г.
1,31.12.2020г.
2,30.12.2020г.
3,30.12.2020г.
4,30.12.2020г.
...,...
445,13.10.2020г.
446,12 октября 2020 г.
447,12.10.2020г.
448,12.10.2020г.


In [33]:
def standartizeDate(d):
    while d.endswith((' ', '.', 'г', 'о', 'д', 'а')):
        d = d[:-1]
    d = d.replace(' октября ', '.10.').replace(' ноября ', '.11.').replace(' декабря ', '.12.')
    return d

In [34]:
for i in range(len(dates)):
    dates[i] = standartizeDate(dates[i])

In [35]:
dates_df = pd.DataFrame(dates, columns=['Дата собрания'])
dates_df

,Дата собрания
0,31.12.2020
1,31.12.2020
2,30.12.2020
3,30.12.2020
4,30.12.2020
...,...
445,13.10.2020
446,12.10.2020
447,12.10.2020
448,12.10.2020


## Форма собрания

In [36]:
types = {
    'заочное голосование': 'заочное голосование',
    'совместное присутствие': 'совместное присутствие',
    'очное собрание': 'совместное присутствие'
}

In [37]:
end_words = (':', 'ента')

def get_form(text, index):
    if index in invalid_texts:
        return invalid_texts[index]['form']
    
    for form_type in types:
        if form_type in text.lower():
            return types[form_type]
        
    return 'тип собрания не определён'

In [38]:
forms = []

for i in range(texts_n):
    forms.append(get_form(texts_df.iloc[i]['text'], i))

In [39]:
forms_df = pd.DataFrame(forms, columns=['Форма собрания'])
forms_df

,Форма собрания
0,заочное голосование
1,заочное голосование
2,заочное голосование
3,заочное голосование
4,заочное голосование
...,...
445,заочное голосование
446,заочное голосование
447,заочное голосование
448,заочное голосование


## Вопрос о выплате дивидендов

In [40]:
dividends = ['вопрос не поднимался'] * texts_n
texts_with_dividends = []

for i in range(texts_n):
    if i in invalid_texts:
        dividends[i] = invalid_texts[i]['dividends']
    for_it_label = re.search('\Wза\W.*?(\d+(?:\s?\d+)*)', texts_df.iloc[i]['text'].lower())
    against_it_label = re.search('против\W.*?(\d+(?:\s?\d+)*)', texts_df.iloc[i]['text'].lower())
    
    if 'дивиденд' in texts_df.iloc[i]['text'].lower() and for_it_label and against_it_label:
        texts_with_dividends.append(i)

print(len(texts_with_dividends))
texts_with_dividends[:10]

84


[1, 2, 8, 12, 13, 35, 38, 40, 41, 57]

In [41]:
def get_dividend(text, index):
    text = text.lower() 
    
    if re.search('\Wза\W.*?единогласн[\s\S]+против', text):
        return 'принято решение выплатить дивиденды'
    
    for_it = int(re.findall('\Wза\W.*?(\d+(?:\s?\d+)*)', text)[0].replace(' ', ''))
    against_it = int(re.findall('против.+?(\d+(?:\s?\d+)*)', text)[0].replace(' ', ''))
    
    return 'принято решение выплатить дивиденды' if for_it > against_it else 'принято решение не выплачивать дивиденды'


for index in texts_with_dividends:
    dividends[index] = get_dividend(texts_df.iloc[index]['text'], index)

In [42]:
dividends_df = pd.DataFrame(dividends, columns=['Вопрос о выплате дивидендов'])
dividends_df

,Вопрос о выплате дивидендов
0,вопрос не поднимался
1,принято решение выплатить дивиденды
2,принято решение выплатить дивиденды
3,вопрос не поднимался
4,вопрос не поднимался
...,...
445,вопрос не поднимался
446,вопрос не поднимался
447,вопрос не поднимался
448,вопрос не поднимался


## Итоговая таблица

In [44]:
final_df = pd.concat([full_names_df, short_names_df, addresses_df, inns_df, ogrns_df, dates_df, forms_df, dividends_df, texts_df['text']], axis=1)
final_df.head(20)

,Полное фирменное наименование эмитента,Сокращенное наименование эмитента,Адрес эмитента,ИНН эмитента,ОГРН эмитента,Дата собрания,Форма собрания,Вопрос о выплате дивидендов,text
0,Иркутское публичное акционерное общество энерг...,"ПАО ""Иркутскэнерго""","664025, г. Иркутск, ул. Сухэ-Батора, 3",3800000220,1023801003313,31.12.2020,заочное голосование,вопрос не поднимался,Решения общих собраний участников (акционеров)...
1,"Публичное акционерное общество ""Новороссийский...","ПАО ""НКХП""","353901, Российская Федерация, Краснодарский кр...",2315014748,1032309077822,31.12.2020,заочное голосование,принято решение выплатить дивиденды,Решения общих собраний участников (акционеров)...
2,"НЕПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""ЦЕНТРТРУБ...","АО ""ЦЕНТРТРУБОПРОВОДСТРОЙ""","101000, Г.МОСКВА, ПЕРЕУЛОК КРИВОКОЛЕННЫЙ, ДО...",7701012060,1027739082062,30.12.2020,заочное голосование,принято решение выплатить дивиденды,Решения общих собраний участников (акционеров)...
3,"НЕПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""ЦЕНТРТРУБ...","АО ""ЦЕНТРТРУБОПРОВОДСТРОЙ""","101000, Г.МОСКВА, ПЕРЕУЛОК КРИВОКОЛЕННЫЙ, ДО...",7701012060,1027739082062,30.12.2020,заочное голосование,вопрос не поднимался,Решения общих собраний участников (акционеров)...
4,Публичное акционерное общество «Русская Акваку...,ПАО «Русская Аквакультура»,"Россия, 121353, г. Москва, ул. Беловежская, д. 4",7816430057,1079847122332,30.12.2020,заочное голосование,вопрос не поднимался,"Сообщение о существенном факте ""Решения общих ..."
5,"Общество с ограниченной ответственностью ""Ред ...","ООО ""Ред Софт""","121205, г. Москва, Территория Сколково Инновац...",9705000373,5147746028216,30.12.2020,совместное присутствие,вопрос не поднимался,Решения общих собраний участников (акционеров)...
6,Публичное акционерное общество Национальный ба...,"Банк ""ТРАСТ"" (ПАО)",г. Москва,7831001567,1027800000480,30.12.2020,заочное голосование,вопрос не поднимался,Решения общих собраний участников (акционеров)...
7,Общество с ограниченной ответственностью «Цент...,"ООО ""ЦГРМ ""ГЕНЕТИКО""","Российская Федерация, город Москва",7736650850,1127747086543,30.12.2020,заочное голосование,вопрос не поднимался,Решения общих собраний участников (акционеров)...
8,"Открытое акционерное общество ""Удмуртнефть""","ОАО ""Удмуртнефть""","426057, Удмуртская республика, г.Ижевск, ул. К...",1831034040,1021801147774,30.12.2020,заочное голосование,принято решение выплатить дивиденды,Решения общих собраний участников (акционеров)...
9,Банк энергетического машиностроения (ЭНЕРГОМАШ...,"ПАО ""Энергомашбанк""",г.Санкт-Петербург,7831000066,1027800001261,30.12.2020,заочное голосование,вопрос не поднимался,Решения общих собраний участников (акционеров)...


In [97]:
with pd.ExcelWriter('output.xlsx') as writer:
    final_df.to_excel(writer, sheet_name='parsed_data')